<a href="https://colab.research.google.com/github/arjunchint/3D-LBM-Fluid-Simulation-CUDA/blob/master/arjun_mistral_hack_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from google.colab import userdata
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents
import re
from googleapiclient.discovery import build
import requests
from bs4 import BeautifulSoup
import collections
import os
BRAVE_API_KEY = "BSAQLuIuBZoMA5TREUU13PwivwYIBJw"

from google.colab import userdata
import textwrap
import pandas as pd
import time

In [ ]:
!pip install mistralai
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [ ]:
!pip install brave-search


In [ ]:
from brave import Brave

In [ ]:
import json
from abc import ABC, abstractmethod
from dataclasses import dataclass

@dataclass
class NodeComponentInput:
    # Define the input class here
    name: str
    instructions: dict
    data: dict

@dataclass
class NodeComponentOutput:
    # Define the output class here
    name: str
    data: dict

class NodeComponent(ABC):
    @abstractmethod
    def compute(self, input: NodeComponentInput) -> NodeComponentOutput:
        pass


In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from google.colab import userdata

class Mistral:
    def __init__(self):
        # Get your Mistral API Key from environment variables
        self.api_key = userdata.get('MISTRAL_API_TOKEN')
        self.model = "mistral-large-latest"
        self.client = MistralClient(api_key=self.api_key)

    def call(self, msg: str):
        """
        Non-streaming call to Mistral API
        """
        messages = [
            ChatMessage(role="user", content=msg)
        ]
        return self.client.chat(model=self.model, messages=messages)

    def get_response(self, msg: str) -> str:
        responseObj = self.call(msg)
        return responseObj.choices[0].message.content


In [ ]:
class WikiTableExtractionNode(NodeComponent):

    def compute(self, input: NodeComponentInput) -> NodeComponentOutput:
        # Loop through input's data["documents"] and retrieve for relevant data
        wiki_url = input.instructions["wiki_url"]
        table=pd.read_html(wiki_url, encoding='utf-8')
        List_of_municipalities_in_California = table[1]
        muni_list = []
        for muni in List_of_municipalities_in_California['Name']['Name'].str.replace('†', '',):
          muni_list.append(muni)

        output = NodeComponentOutput(name='Wiki Parsed Table', data={'Municipality_List': muni_list})
        return output

In [ ]:
class GoogleSearchNode(NodeComponent):
    def __init__(self):
        self.mistral = Mistral()
        self.GOOGLE_API_KEY = userdata.get('GOOGLE_SEARCH_API_KEY')
        # Replace with your Custom Search Engine ID (find it in your Google Custom Search settings)
        self.SEARCH_ENGINE_ID = 'b6ccd126391e94d17'

    def compute(self, input: NodeComponentInput) -> NodeComponentOutput:
        # Loop through input's data["documents"] and retrieve for relevant data
        def google_search(search_term, **kwargs):
          service = build("customsearch", "v1", developerKey=self.GOOGLE_API_KEY)
          res = service.cse().list(q=search_term, cx=self.SEARCH_ENGINE_ID, **kwargs).execute()
          return res['items']
        iteration_list = input.data["Municipality_List"]
        search_key_words = input.instructions["search_key_words"]
        search_delay = instructions["search_delay"]
        debug_print = input.instructions["debug"]
        output_links = collections.defaultdict(list)

        for iteration_term in iteration_list:
          results = google_search(' '.join([iteration_term + search_key_words]))
          output_links[iteration_term] = [res['link'] for res in results]
          # Print the title and link of the first few results
          if debug_print:
            for result in results[:3]:
              print(f"{iteration_term} Google results: {result['title']}: {result['link']}")
          if search_delay:
            time.sleep(search_delay)
        output = NodeComponentOutput(name='Search Result Links List Per Muni', data={'Search_Result_Links_List_Per_Muni': output_links})
        return output


In [ ]:
brave = Brave(api_key=BRAVE_API_KEY)
search_terms = "https://www.mountainview.gov/our-city/departments/finance-and-administrative-services/accounting"
search_results = brave.search(q=search_terms, count=10, raw=True)
search_results

{'query': {'original': 'https://www.mountainview.gov/our-city/departments/finance-and-administrative-services/accounting',
  'show_strict_warning': False,
  'is_navigational': False,
  'local_decision': 'drop',
  'local_locations_idx': 0,
  'is_news_breaking': False,
  'spellcheck_off': True,
  'country': 'us',
  'bad_results': False,
  'should_fallback': False,
  'postal_code': '',
  'city': '',
  'header_country': '',
  'more_results_available': True,
  'state': ''},
 'mixed': {'type': 'mixed',
  'main': [{'type': 'web', 'index': 0, 'all': False},
   {'type': 'web', 'index': 1, 'all': False},
   {'type': 'web', 'index': 2, 'all': False},
   {'type': 'web', 'index': 3, 'all': False},
   {'type': 'web', 'index': 4, 'all': False},
   {'type': 'web', 'index': 5, 'all': False},
   {'type': 'web', 'index': 6, 'all': False},
   {'type': 'web', 'index': 7, 'all': False},
   {'type': 'web', 'index': 8, 'all': False},
   {'type': 'web', 'index': 9, 'all': False},
   {'type': 'web', 'index': 10

In [ ]:
search_results['web']['results'][0]['url']

'https://www.mountainview.gov/our-city/departments/finance-and-administrative-services'

In [ ]:
class BraveSearchNode(NodeComponent):
    def __init__(self):
        self.brave = Brave(api_key=BRAVE_API_KEY)

    def compute(self, input: NodeComponentInput) -> NodeComponentOutput:
        # Loop through input's data["documents"] and retrieve for relevant data

        def brave_search(self, search_terms, **kwargs):
          search_results = self.brave.search(q=search_terms, count=3, raw=True)
          return [result['url'] for result in search_results['web']['results']]
        iteration_list = input.instructions["for_each_list"]
        search_key_words = input.instructions["search_key_words"]
        debug_print = input.instructions["debug_print"]

        for iteration_term in iteration_list:
          results = brave_search(' '.join([iteration_term + search_key_words]))

        # Print the title and link of the first few results
        if debug_print:
          for result in results[:3]:
            print(f"{muni} Google results: {result['title']}: {result['link']}")
        output = NodeComponentOutput(name='Wiki Parsed Table', data={'Search_Result_Links_List': [res['link'] for res in results]})
        return output










In [ ]:
class LLMGuidedPageCrawlNode(NodeComponent):
    def __init__(self):
        self.mistral = Mistral()

    def compute(self, input: NodeComponentInput) -> NodeComponentOutput:
      def is_site_pdf(url):
        headers = {
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive',
                'refere': 'https://www.google.com',
                'cookie': """your cookie value ( you can get that from your web page) """
            }
        response = requests.get(url, headers=headers)
        content_type = response.headers.get('content-type')

        if 'application/pdf' in content_type:
            return True
        elif 'text/html' in content_type:
            return False
        return False
      def extract_relevant_links_from_muni_website(url):
          """Extracts links from a webpage containing text related to
            'Annual Comprehensive Financial Report'

          Args:
              url: The URL of the webpage to analyze.

          Returns:
              A list of dictionaries, each containing:
                  'text': The associated text of the link.
                  'href': The link's URL (href attribute).
          """
          headers = {
                  'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36',
                  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
                  'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                  'Accept-Encoding': 'none',
                  'Accept-Language': 'en-US,en;q=0.8',
                  'Connection': 'keep-alive',
                  'refere': 'https://www.google.com',
                  'cookie': """your cookie value ( you can get that from your web page) """
              }
          try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Raise an error for bad HTTP status codes
            soup = BeautifulSoup(response.content, 'html.parser')
            body = soup.find('body')
            # print(body)

            relevant_links = {}
            search_phrases = [
                "annual comprehensive financial report",
                "acfr",  # Add more variations if needed
            ]

            # Search in link text, headers, and paragraphs
            report_link = soup.find(lambda tag: tag.name == "a" and "Annual Comprehensive Financial Reports" in tag.text)
            for element in body.find_all(['a', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'div']):
                text = element.get_text(strip=True).lower()
                # print(text)
                if any(phrase.lower() in text for phrase in search_phrases):
                    if element.name == 'a' and element.get('href'):
                        relevant_links[element['href']] = {
                            'text': text[:100],
                            'href': element['href']
                        }
                    else:  # Assume it's a header or paragraph with a nearby link
                        link = element.find_next('a')
                        if link  and link.get('href'):
                            relevant_links[link['href']] = {
                                'text': text[:100],
                                'href': link['href']
                            }
            # print(relevant_links)
            for key, result in relevant_links.items():
              print(f"{muni} Site Link Parsing: {result['text']}: {result['href']}")
            return [elements for link, elements in relevant_links.items()]
          except:
            return []
      def linkmap_to_string(links):
        out = []
        for link in links:
            if not (link['href'].startswith('/') or link['href'].startswith('http')):
              continue
            out.append(f"{link['href']}: {link['text'][:100]} ")
        return '\n'.join(out)

      def ask_llm_for_correct_ACFR_link(mistral_, LLM_site_search_primpt, prompt_suffix):
        return mistral_.get_response(LLM_site_search_primpt + ' ' + prompt_suffix)

      def find_urls_in_text(text):
        regex=r"\b((?:https?://)?(?:(?:www\.)?(?:[\da-z\.-]+)\.(?:[a-z]{2,6})|(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)|(?:(?:[0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,7}:|(?:[0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,5}(?::[0-9a-fA-F]{1,4}){1,2}|(?:[0-9a-fA-F]{1,4}:){1,4}(?::[0-9a-fA-F]{1,4}){1,3}|(?:[0-9a-fA-F]{1,4}:){1,3}(?::[0-9a-fA-F]{1,4}){1,4}|(?:[0-9a-fA-F]{1,4}:){1,2}(?::[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:(?:(?::[0-9a-fA-F]{1,4}){1,6})|:(?:(?::[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(?::[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(?:ffff(?::0{1,4}){0,1}:){0,1}(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])|(?:[0-9a-fA-F]{1,4}:){1,4}:(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9]))|(?:/[\w\.-]*))(?::[0-9]{1,4}|[1-5][0-9]{4}|6[0-4][0-9]{3}|65[0-4][0-9]{2}|655[0-2][0-9]|6553[0-5])?(?:/[\w\.-]*)*/?)\b"
        matches = re.findall(regex, text)
        return matches
      def create_output_pdf_link(row):
          if row['Parsed LLM Output'] == 'LLM Skipped':
              val = row['Google Search Result 1']
          elif row['Parsed LLM Output'].startswith('/'):
              val = str(os.path.dirname(row['Google Search Result 1'])) + row['Parsed LLM Output']
          else:
              val = row['Parsed LLM Output']
          return val

      Search_Result_Links_List = input.data["Search_Result_Links_List_Per_Muni"]
      Output_Filepath = input.instructions["output_filepath"]
      debug_print = input.instructions["debug"]
      LLM_site_search_primpt = input.instructions["LLM_site_search_prompt"]

      results = []
      for muni, search_links in Search_Result_Links_List.items():
        for result_url in search_links:
          if is_site_pdf(result_url):
            llm_prompt_structured_links = 'LLM Skipped'
            possible_llm_links = 'LLM Skipped'
            parsed_llm_link = 'LLM Skipped'
            break
          links_on_muni_site = extract_relevant_links_from_muni_website(result_url)
          if not links_on_muni_site:
            llm_prompt_structured_links = 'LLM Skipped'
            possible_llm_links = 'LLM Skipped'
            parsed_llm_link = 'LLM Skipped'
            continue
          llm_prompt_structured_links = linkmap_to_string(links_on_muni_site)
          possible_llm_links = ask_llm_for_correct_ACFR_link(self.mistral, LLM_site_search_primpt, llm_prompt_structured_links)
          parsed_llm_link = find_urls_in_text(possible_llm_links)
          if parsed_llm_link:
            parsed_llm_link = parsed_llm_link[0]
            break
          else:
            parsed_llm_link = 'Unable to parse links with LLM'
            continue
        result = {
            'Municipality': muni,
            'Google Search Result 1': search_links[0],
            'Google Search Result 2': search_links[1],
            'Google Search Result 3': search_links[2],
            'Structured Links on Muni Site': llm_prompt_structured_links,
            'LLM Output': possible_llm_links,
            'Parsed LLM Output': parsed_llm_link
        }
        if debug_print:
          print(result)
        results.append(result)
      final_results = pd.DataFrame(results)
      final_results['url'] = final_results.apply(create_output_pdf_link, axis=1)
      if debug_print:
        pd.set_option('display.max_rows', None)
        pd.set_option('max_colwidth', None)
        print(final_results)
      # final_results.to_csv(Output_Filepath)
      output_dict = final_results.to_dict('records')
      output = NodeComponentOutput(name='Parsed Records', data={'output_dict': output_dict})
      return output


In [ ]:

instructions = {}
instructions["debug"] = True
instructions["search_key_words"] = "\"City of\" California Latest Annual Comprehensive Financial Report"
instructions["search_delay"] = 2
instructions["wiki_url"] = "https://en.wikipedia.org/wiki/List_of_municipalities_in_California"
instructions["LLM_site_search_prompt"] = "Please output just the link associated with the municipality's latest Annual Comprehensive Financial Report from this mapping of href links and associated text: "
instructions["output_filepath"] = 'municipal_audit_links.csv'




In [ ]:
WikiExtractor_input = NodeComponentInput(name="Starting Node", instructions=instructions, data=None)
WikiExtractor = WikiTableExtractionNode()
WikiOutput = WikiExtractor.compute(WikiExtractor_input)


In [ ]:
WikiOutput

NodeComponentOutput(name='Wiki Parsed Table', data={'Municipality_List': ['Adelanto', 'Agoura Hills', 'Alameda', 'Albany', 'Alhambra', 'Aliso Viejo', 'Alturas', 'Amador City', 'American Canyon', 'Anaheim', 'Anderson', 'Angels Camp', 'Antioch', 'Apple Valley', 'Arcadia', 'Arcata', 'Arroyo Grande', 'Artesia', 'Arvin', 'Atascadero', 'Atherton', 'Atwater', 'Auburn', 'Avalon', 'Avenal', 'Azusa', 'Bakersfield', 'Baldwin Park', 'Banning', 'Barstow', 'Beaumont', 'Bell', 'Bell Gardens', 'Bellflower', 'Belmont', 'Belvedere', 'Benicia', 'Berkeley', 'Beverly Hills', 'Big Bear Lake', 'Biggs', 'Bishop', 'Blue Lake', 'Blythe', 'Bradbury', 'Brawley', 'Brea', 'Brentwood', 'Brisbane', 'Buellton', 'Buena Park', 'Burbank', 'Burlingame', 'Calabasas', 'Calexico', 'California City', 'Calimesa', 'Calipatria', 'Calistoga', 'Camarillo', 'Campbell', 'Canyon Lake', 'Capitola', 'Carlsbad', 'Carmel-by-the-Sea', 'Carpinteria', 'Carson', 'Cathedral City', 'Ceres', 'Cerritos', 'Chico', 'Chino', 'Chino Hills', 'Chowchi

In [ ]:
GoogleSearch = GoogleSearchNode()
GoogleSearchInput = NodeComponentInput(name="Google Search Node", instructions=instructions, data=WikiOutput.data)
GoogleSearchOutput = GoogleSearch.compute(GoogleSearchInput)



Adelanto Google results: Adelanto, CA: https://ci.adelanto.ca.us/services/index.php
Adelanto Google results: 2022-23 Annual Comprehensive Financial Report: https://www.calpers.ca.gov/docs/forms-publications/acfr-2023.pdf
Adelanto Google results: Annual Reports | Victorville, CA: https://www.victorvilleca.gov/government/city-departments/administrative-services/finance/accounting-administration/prior-years
Agoura Hills Google results: Annual Comprehensive Finance Report | City of Agoura Hills, CA: https://www.agourahillscity.org/department/finance-information-technology/comprehensive-annual-finance-report
Agoura Hills Google results: Fiscal Services/Accounting | City of Simi Valley, CA: https://www.simivalley.org/departments/administrative-services/fiscal-services-accounting
Agoura Hills Google results: Positions and Salaries | City of Agoura Hills, CA: https://www.agourahillscity.org/department/human-resources/positions-and-salaries
Alameda Google results: Finance: https://www.alamedaca

In [ ]:
LLMGuidedPageCrawl = LLMGuidedPageCrawlNode()
LLMGuidedPageCrawlInput = NodeComponentInput(name="LLMGuidedPageCrawl Node", instructions=instructions, data=GoogleSearchOutput.data)
LLMGuidedPageCrawlOutput = LLMGuidedPageCrawl.compute(LLMGuidedPageCrawlInput)



Streaming output truncated to the last 5000 lines.
Paso Robles Site Link Parsing: homegovernmentdepartmentsadministrative servicesfinancefinancial reportsfinancial reportsfy 2023/24 : /
Paso Robles Site Link Parsing: fy 2023/24 budget reports, associated council staff reports and presentationsfiscal year 2023/24coun: https://www.prcity.com/ArchiveCenter/ViewFile/Item/1215
Paso Robles Site Link Parsing: adoption of the budgettheadopted budget for fy 2021-22 (pdf)was adopted by city council at the june : https://www.prcity.com/DocumentCenter/View/32138/Adopted-Budget-FY-2021-22-PDF
Paso Robles Site Link Parsing: acfr: https://www.prcity.com/Archive.aspx?AMID=36
Paso Robles Site Link Parsing: most recent acfr (pdf): https://www.prcity.com/DocumentCenter/View/37458/Annual-Comprehensive-Financial-Report---FY-2022-23
Paso Robles Site Link Parsing: beginning with fiscal year ending june 30, 1999, the city of paso robles has annually received a cer: https://www.prcity.com/DocumentCenter/View/3